In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s [%(process)d] %(levelname)-8s "
                    "%(name)s,%(lineno)s\t%(message)s")
logging.getLogger().setLevel('INFO')

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from tqdm import tqdm_notebook as tqdm

In [ ]:
# Read information to connect to the database and put it in environment variables
import os
with open('../ENVVARS.txt') as f:
    for line in f:
        parts = line.split('=')
        if len(parts) == 2:
            os.environ[parts[0]] = parts[1].strip()

In [ ]:
db_name = 'ticclat'
os.environ['dbname'] = db_name

In [ ]:
import os
from ticclat.dbutils import create_ticclat_database

create_ticclat_database(dbname=os.environ['dbname'], user=os.environ['user'], passwd=os.environ['password'])

In [ ]:
from ticclat.ticclat_schema import Lexicon, Wordform, Anahash, Document, Corpus, WordformLink, WordformLinkSource

from ticclat.dbutils import get_session, session_scope

Session = get_session(os.environ['user'], os.environ['password'], os.environ['dbname'])

In [ ]:
elex_words_file = '/home/jvdzwaan/data/ticclat/TICCLAT/elex/e-Lex-1.1.uniq.utf8.txt'
elex_lemma_file = '/home/jvdzwaan/data/ticclat/TICCLAT/elex/e-Lex-1.1.lemma_wordform.utf8.nonumbers.tsv'

l_wf_pairs = pd.read_csv(elex_lemma_file, sep='\t', header=None)
l_wf_pairs.columns = ['lemma', 'variant']
print(l_wf_pairs.shape)
l_wf_pairs.head()

In [ ]:
%%time
from ticclat.dbutils import add_lexicon_with_links

with session_scope(Session) as session:
    name = 'e-Lex-1.1.lemma_wordform.utf8.nonumbers'
    vocabulary = True
    from_column = 'lemma'
    from_correct = True
    to_column = 'variant'
    to_correct = True
    
    add_lexicon_with_links(session, name, vocabulary, l_wf_pairs, from_column, to_column, from_correct, to_correct)